# Portfolio Optimization Example

This notebook demonstrates simple allocation workflows using return and covariance estimates.

In [ ]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
from bist_quant import DataLoader

loader = DataLoader()
prices_raw = loader.load_prices()

In [ ]:
if {"Date", "Ticker", "Close"}.issubset(prices_raw.columns):
    prices = prices_raw.pivot(index="Date", columns="Ticker", values="Close").sort_index()
else:
    prices = prices_raw.copy()

returns = prices.pct_change().dropna(how="all")
returns = returns.dropna(axis=1, how="any")
returns = returns.iloc[:, : min(20, returns.shape[1])]

mu = returns.mean() * 252
cov = returns.cov() * 252
n = len(mu)

print(f"Using {n} assets")

In [ ]:
def neg_sharpe(weights, mu, cov, rf=0.0):
    port_ret = weights @ mu.values
    port_vol = np.sqrt(weights @ cov.values @ weights)
    if port_vol == 0:
        return 1e6
    return -(port_ret - rf) / port_vol

x0 = np.ones(n) / n
bounds = [(0.0, 1.0)] * n
constraints = [{"type": "eq", "fun": lambda w: np.sum(w) - 1.0}]

opt = minimize(neg_sharpe, x0, args=(mu, cov), method="SLSQP", bounds=bounds, constraints=constraints)
weights = pd.Series(opt.x, index=mu.index).sort_values(ascending=False)
weights.head(10)

In [ ]:
eq_weights = np.ones(n) / n
mv_weights = opt.x

def annual_stats(weights):
    r = float(weights @ mu.values)
    v = float(np.sqrt(weights @ cov.values @ weights))
    return r, v

eq_ret, eq_vol = annual_stats(eq_weights)
mv_ret, mv_vol = annual_stats(mv_weights)

pd.DataFrame({
    "strategy": ["equal_weight", "max_sharpe"],
    "annual_return": [eq_ret, mv_ret],
    "annual_vol": [eq_vol, mv_vol],
})